In [ ]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from nuclass.data import icecat1, icecat1_cut_path
from astropy.coordinates import SkyCoord, get_sun
from astropy.time import Time

# Neutrino Data

 We use a catalogue of high-energy neutrinos from IceCuce known as ICECAT1.
The catalogue is downloaded from, and described at https://icecube.wisc.edu/data-releases/2023/04/icecat-1-the-icecube-event-catalog-of-alert-tracks/ . It contains neutrinos through to the end of 2023.

You can find the latest list of neutrino alerts on the GCN page https://gcn.gsfc.nasa.gov/amon_icecube_gold_bronze_events.html , and sign up to receive notifications or access via API.

In [ ]:
icecat1

In [ ]:
list(x for x in icecat1.columns)

# Let's explain what some of these fields are...

(**Copied directly from IceCube's documentation**)

For each event, the CSV table contains:
 - RUNID,EVENTID: Unique RunID and EventID combination from IceCube DAQ system
 - START,EVENTMJD: Date/time of event detection
:q
 - I3TYPE: Identification of event selection type (see supporting paper publication for details).  gfu-gold, gfu-bronze, ehe-gold, hese-gold, or hese-bronze types
 - OTHER_I3TYPES: List of other I3TYPE event selection types this event additionally passed.
 - RA,DEC [deg] (and _ERR): Best fit direction in J2000 equatorial coordinates, with asymmetric 90% CL error rectangle boundaries.
 - ENERGY:[TeV] Most probable neutrino energy that would have produced this event.  Calculated assuming an E^(-2.19) astrophysical neutrino power law flux.
 - FAR: [yr^(-1)] Rate of background events expected for alert events at this energy and sky location.
 - SIGNAL: Probability event is of astrophysical origin, calculated assuming an E^(-2.19) astrophysical neutrino power law flux.
 - *_SCR: Probabilities from post-alert convolutional neural network based classifier applied to each event to  better distinguish each events topological signal in the detector
     - THRGOING_SCR: Primary event vertex outside is the detector and a muon-like track is observed passing through the instrumented volume
     - START_SCR: Primary event vertex is inside the instrumented volume and a muon-like track is seen
     - CASCADE_SCR: Primary event vertex is inside the instrumented volume and a shower (non-muon-like track) is observed
     - SKIMMING_SCR: Primary event vertex outside is the detector and little or no energy deposited within instrumented volume
     - STOP_SCR: Primary event vertex outside is the detector and a muon-like track is observed stopping in the instrumented volume
 - CR_VETO [Bool]: Significant in-time cosmic-ray shower activity detected in the surface IceTop array, indicating this event is likely a background event.  
     - Note: at time of alert creation this was not available, and therefore events have been evaluated and tagged after fact rather than removed. 

**An additional note about names**

IceCube neutrinos are named in the same way as GRBs:

    IC + date of detection + alphabetical order key

So the first neutrino detected on 2024 September 01 would be IC240901A, the second would be IC240901B, and so on.

# Understanding the neutrino alert data

IceCube's effective area varies as a function of declination, and azimuth. 

However, because the Earth rotates and IceCube is at the South Pole, azimuthal variations get smoothed out over time. Zenith variations do not. 

In [ ]:
# Plot the distribution of alerts as a function of sin(declination). Is it uniform?

# Remember to convert from degrees to radians first!
sindecs = # Your answer goes here!!!

plt.hist(sindecs) # Sin(dec) not dec to account for projection effects for a sphere

# Plot median, and 25th/75th percentile 
for val in [25, 50., 75.]:
    plt.axvline(np.percentile(sindecs, val, axis=0), linestyle=":", color="k")

plt.xlabel("Sin(declination)")
plt.xlim(-1., 1.)

In [ ]:
# Plot RA, is it uniform?

# Augmenting Neutrino Data

## Localisation Area

In [ ]:
# Let's approximate the localisation from IceCube as a rectangle. 
# The true contour is not always Gaussian/elliptical, and is not released in real time.
# Generally the simplest thing is to just take the rectangle at face value.

# What is the area of each alert? Remember to account for projection on a sphere - https://en.wikipedia.org/wiki/Great-circle_distance

area = # Your answer goes here

icecat1["AREA"] = area

In [ ]:
# What is the median localisation?
print(f'Median neutrino area is {icecat1["AREA"].median():.1f} sq. deg, mean is {icecat1["AREA"].mean():.1f}')

In [ ]:
# Plot histogram of areas
plt.hist(np.log10(icecat1["AREA"]))
plt.xlabel("Area [sq deg]")
x_ticks = np.logspace(-1, 3, 5) # Convert labels from log
plt.xticks(np.log10(x_ticks), x_ticks)
plt.axvline(np.log10(icecat1["AREA"].median()), color="k", linestyle=":")

The neutrino localisations vary dramatically. The median is ~8 sq deg, but some have localisations > 100 sq deg. 
This is due to the topology of each event, with some neutrinos being much easier to reconstruct than others. The probability that an object accidentally coincides with a neutrino is much higher for large neutrinos. 

In [ ]:
# Now make some basic cuts to select "high-quality alerts"

mask = (
    # Your answer goes here!!!
    # Cut to remove CR-coincident events
    # Cut those events with large area (< 35 sq. deg.)
    # Cut those events with signal probability < 50%
)

print(f"Selected {mask.sum()} neutrinos out of {len(mask)} total neutrinos")

In [ ]:
icecat1[mask].to_csv(icecat1_cut_path)